In [0]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

schema = StructType(
    [
        StructField("test_name", StringType(), True),
        StructField("result", StringType(), True),
        StructField("test_description", StringType(), True),
        StructField("meta_data", ArrayType(StringType()), True),
    ]
)

all_test_results = spark.createDataFrame([], schema)

    # Append the test result to all_test_results 


# Meta data checks

- **Table Name Test :**  
TableExists(catalogName, schemaName, tableName, test_name=None, test_description=None,meta_data=None) 
- **Column Name Test :**   
compare_attributescompare_attributes(Attributes_in_stm_list, table_df, test_name=None, test_description=None,meta_data=None):

- **Data type test :**

- **oprerational_atttibute test :** 
check_attributes_presence(attributes_list, table_df, test_name=None, test_description=None,meta_data=None)

In [0]:
def compare_attributes(
    Attributes_in_stm_list,
    table_df,
    test_name="Column name test",
    test_description=None,
    meta_data=None,
):

    test_description = (
        "Compare the column names in STM against delta tables in data bricks"
    )
    # Get the list of attributes present in Store_count_report_raw
    attributes_in_table_df = table_df.columns

    # Compare the two lists and identify missing or extra attributes
    missing_attributes = set(Attributes_in_stm_list) - set(attributes_in_table_df)
    extra_attributes = set(attributes_in_table_df) - set(Attributes_in_stm_list)

    # Determine the test result based on the comparison
    if not missing_attributes and not extra_attributes:
        test_result = "PASS"
    else:
        test_result = "FAIL"

    # Display the reason for failure
    if test_result == "FAIL":
        failure_reason = {
            "missing_attributes": list(missing_attributes),
            "extra_attributes": list(extra_attributes),
        }
        display(failure_reason)
    else:
        display(test_result)

    # Append the test result to all_test_results DataFrame
    global all_test_results
    new_result = [(test_name, test_result, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)
    all_test_results = all_test_results.union(new_result_df)


# compare_attributes(fuel_master_location_attributes, df_fuel_master_location)
# compare_attributes(electricity_master_location_attributes, df_electricity_master_location)

In [0]:
from pyspark.sql.types import *


# Function to validate if the column and its data type are matching and push the result to all_test_results
def validate_schema(
    df,
    expected_schema,
    test_name="Schema Validation",
    test_description="Validate schema of the DataFrame",
    meta_data=None,
):
    mismatched_attributes = []

    for column, expected_type in expected_schema.items():
        if column in df.schema.names:
            actual_type = df.schema[column].dataType
            if not isinstance(actual_type, type(expected_type)):
                mismatched_attributes.append(
                    (column, str(expected_type), str(actual_type))
                )
        else:
            mismatched_attributes.append(
                (column, str(expected_type), "Column not found")
            )

    if mismatched_attributes:
        result = "FAIL"
        mismatched_df = spark.createDataFrame(
            mismatched_attributes,
            ["column_name", "expected_data_type", "actual_data_type"],
        )
        display(mismatched_df.filter("expected_data_type != actual_data_type"))
    else:
        result = "PASS"

    display(result)

    global all_test_results
    new_result = [(test_name, result, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)
    all_test_results = all_test_results.union(new_result_df)


# Example usage
# validate_schema(df_fuel_master_location, expected_schema, "Schema Validation", "Validate schema of df_fuel_master_location")

In [0]:
def check_attributes_presence(
    table_df,
    attributes_list,
    test_name="Attribute presence test",
    test_description=None,
    meta_data=None,
):

    test_description = (
        "Check if all attributes from the list are present in the DataFrame"
    )

    # Get the list of attributes present in the DataFrame
    attributes_in_table_df = table_df.columns

    # Identify missing attributes
    missing_attributes = set(attributes_list) - set(attributes_in_table_df)

    # Determine the test result based on the presence of attributes
    test_result = "PASS" if not missing_attributes else "FAIL"

    # Display the reason for failure
    if test_result == "FAIL":
        failure_reason = {"missing_attributes": list(missing_attributes)}
        display(failure_reason)
    else:
        display(test_result)

    # Append the test result to all_test_results DataFrame
    global all_test_results
    new_result = [(test_name, test_result, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)
    all_test_results = all_test_results.union(new_result_df)


# Example usage
# check_attributes_presence(fuel_master_location_attributes, df_fuel_master_location)
# check_attributes_presence(electricity_master_location_attributes, df_electricity_master_location)

# Data Checks

In [0]:
def check_row_count_df(
    df1, df2, test_name="Row count test", test_description=None, meta_data=None
):

    test_description = "Check if the row counts of two DataFrames are equal"

    # Get the row counts of both DataFrames
    count_df1 = df1.count()
    count_df2 = df2.count()

    # Determine the test result based on the row counts
    test_result = "PASS" if count_df1 == count_df2 else "FAIL"

    # Display the reason for failure
    if test_result == "FAIL":
        failure_reason = {"count_df1": count_df1, "count_df2": count_df2}
        display(failure_reason)
    else:
        display(test_result)

    # Append the test result to all_test_results DataFrame
    global all_test_results
    new_result = [(test_name, test_result, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)
    all_test_results = all_test_results.union(new_result_df)


# Example usage
# check_row_count(df1, df2)

In [0]:
def check_row_count(
    df,
    expected_count,
    test_name="Row count test",
    test_description=None,
    meta_data=None,
):
    test_description = (
        "Check if the row count of the DataFrame matches the expected count"
    )

    # Get the row count of the DataFrame
    actual_count = df.count()

    # Determine the test result based on the row count
    test_result = "PASS" if actual_count == expected_count else "FAIL"

    # Display the reason for failure
    if test_result == "FAIL":
        failure_reason = {
            "actual_count": actual_count,
            "expected_count": expected_count,
        }
        display(test_result)
        display(failure_reason)
    else:
        display(test_result)

    # Append the test result to all_test_results DataFrame
    global all_test_results
    new_result = [(test_name, test_result, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)
    all_test_results = all_test_results.union(new_result_df)


# Example usage
# check_row_count(df, 100)

In [0]:
def compare_total_vs_distinct(
    df,
    attributes,
    test_name="Distinct value test",
    test_description=None,
    meta_data=None,
):
    global all_test_results  # Declare all_test_results as global at the beginning

    # Get the total count of rows for the given DataFrame
    total_count = df.count()

    # Get the total count of distinct rows for the given DataFrame based on the specified attributes
    distinct_count = df.select(attributes).distinct().count()

    # Display the results
    display(
        f"Total Rows: {total_count}, Distinct Rows based on {attributes}: {distinct_count}"
    )

    # Check if the total count and distinct count match
    test_result = "PASS" if total_count == distinct_count else "FAIL"
    display(test_result)

    # Append the test result to all_test_results DataFrame

    test_description = (
        f"Check for distinct values based on {attributes} in the given DataFrame"
    )
    new_result = [(test_name, test_result, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)

    all_test_results = all_test_results.union(new_result_df)


# Ensure all_test_results DataFrame is initialized before calling the func

In [0]:
def primary_key_check(
    df,
    primary_key_columns,
    group_by_columns=None,
    test_name="Primary Key Check",
    test_description=None,
    meta_data=None,
):
    global all_test_results  # Declare all_test_results as global at the beginning

    # If group_by_columns is provided, group the DataFrame by those columns
    if group_by_columns:
        df = df.groupBy(group_by_columns).agg(*[col(c) for c in primary_key_columns])

    # Get the total count of rows for the given DataFrame
    total_count = df.count()

    # Get the total count of distinct rows for the given DataFrame based on the primary key columns
    distinct_count = df.select(primary_key_columns).distinct().count()

    # Check if the total count and distinct count match
    test_result = "PASS" if total_count == distinct_count else "FAIL"
    display(
        f"Total Rows: {total_count}, Distinct Rows based on {primary_key_columns}: {distinct_count}"
    )
    display(test_result)

    # Append the test result to all_test_results DataFrame
    test_description = f"Check for primary key uniqueness based on {primary_key_columns} in the given DataFrame"
    new_result = [(test_name, test_result, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)

    all_test_results = all_test_results.union(new_result_df)


# Example usage
# primary_key_check(df_electricity_master_location, ["electricity_consumption_uuid", "electricity_location_nbr"], ["brand_nm"])

In [0]:
%python
from pyspark.sql.functions import col, count, when, isnull

def check_null_values(input_df, non_null_attributes, test_name="Null value test", test_description=None, meta_data=None):
    # Select the specified columns and count nulls for each column in the input DataFrame
    null_counts_df = input_df.select(
        [count(when(isnull(col(c)), c)).alias(c) for c in non_null_attributes]
    )

    # Filter columns with null values and display the column name and null count
    null_counts_filtered = null_counts_df.select(
        [col(c) for c in null_counts_df.columns if null_counts_df.select(col(c)).first()[0] > 0]
    )

    # Check if null_counts_filtered is empty
    is_empty = len(null_counts_filtered.columns) == 0

    display(null_counts_filtered)
    display(null_counts_df)

    # Append the test result to all_test_results DataFrame
    
    test_description = "Check for null values in specified columns of the input DataFrame"
    test_result = "PASS" if is_empty else "FAIL"
    
    display(test_result)
    
    global all_test_results
    new_result = [(test_name, test_result, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)
    all_test_results = all_test_results.union(new_result_df)



In [0]:
def check_data_completeness(
    df,
    accepted_values,
    test_name="Data Completeness Check",
    test_description=None,
    meta_data=None,
):
    global all_test_results  # Declare all_test_results as global at the beginning
    results = []
    for column, values in accepted_values.items():
        if column in df.columns:
            actual_values = df.select(column).distinct().collect()
            actual_values = [
                str(row[column]) if row[column] is not None else "null"
                for row in actual_values
            ]
            status = "PASS" if set(actual_values).issubset(set(values)) else "FAIL"
            results.append(
                Row(
                    column=column,
                    status=status,
                    actual_values=actual_values,
                    accepted_values=values,
                )
            )

    # Append the test results to all_test_results DataFrame
    overall_status = "PASS" if all(row.status == "PASS" for row in results) else "FAIL"
    display(overall_status)

    test_description = f"Check for data completeness based on accepted values in {df}"
    new_result = [(test_name, overall_status, test_description, meta_data)]
    new_result_df = spark.createDataFrame(new_result, schema=all_test_results.schema)
    all_test_results = all_test_results.union(new_result_df)

    result_df = spark.createDataFrame(results)
    display(result_df)

In [0]:
from pyspark.sql.functions import col


# Function to display distinct values for the specified columns in a DataFrame
def display_distinct_values(
    df,
    df_name,
    additional_columns=None,
    test_name=None,
    test_description=None,
    meta_data=None,
):
    for column in df.columns:
        distinct_values = df.select(column, *additional_columns).distinct()
        print(f"Distinct values for {column} in {df_name}:")
        display(distinct_values)

In [0]:
print("Validation dependencies are loaded Successfully!!")

Validation dependencies are loaded Successfully!!
